In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import mne
from pqdm.processes import pqdm
import stat_func
import utilities

mne.set_log_level(verbose='WARNING')

N_JOBS = 1
# N_JOBS = 32

%matplotlib qt

## Let's look at one file

In [2]:
subject = "04"
meg_subject_dir = Path(f'lb_analysis_sparse/import_metadata/sub-{subject}/meg/')
epochs_fname = meg_subject_dir / f"sub-{subject}_task-sparse_metadata_proc-clean_epo.fif"
epochs = mne.read_epochs(epochs_fname)

In [3]:
# epochs.event_id

In [4]:
epochs.average().plot_joint();

In [5]:
epochs.metadata

,event_name,Node,Tone,Comm,Transition,numTrial,numBlock,numTrialinBlock,Cond
0,Node10/Tone7/Comm2/Transition1,10,7,2,1,1,0,1,1
1,Node11/Tone8/Comm2/Transition1,11,8,2,1,2,0,2,1
2,Node12/Tone3/Comm2/Transition2,12,3,2,2,3,0,3,1
3,Node11/Tone8/Comm2/Transition2,11,8,2,2,4,0,4,1
5,Node9/Tone1/Comm2/Transition2,9,1,2,2,6,0,6,1
...,...,...,...,...,...,...,...,...,...
6715,Node5/Tone5/Comm1/Transition1,5,5,1,1,6716,6,956,1
6716,Node6/Tone11/Comm1/Transition2,6,11,1,2,6717,6,957,1
6717,Node5/Tone5/Comm1/Transition2,5,5,1,2,6718,6,958,1
6718,Node3/Tone2/Comm1/Transition3,3,2,1,3,6719,6,959,2


In [6]:
epochs.metadata['Cond'].unique()

array([1, 3, 2, 4])


## Compute and save contrast

In [7]:

def compute_contrast(subject) :
    # bids_path = '/neurospin/unicog/protocols/MEG/SparseComm_LucasBenjamin_2021/BIDS_MEG/bids_data_all/'
    meg_subject_dir = Path(f'lb_analysis_sparse/import_metadata/sub-{subject}/meg/')
    outputdir = Path(f'lb_analysis_sparse/ERP_EarlyContrast/sub-{subject}/meg/')
    base_filename = f'sub-{subject}_task-sparse_metadata'

    # Import epochs
    extension = '_proc-clean_epo'
    filename = base_filename + extension
    fname_in = meg_subject_dir / (filename + '.fif')
    print('**** sub_', subject, ': Reading epochs')
    epochs = mne.read_epochs(fname_in, preload=True)

    # Compute Contrast
    contrast = [0, 1, 0, -1]
    all_dat = [epochs[f'Cond=={i}'].average() for i in range(1, 5)]
    res = mne.combine_evoked(all_dat, weights=contrast)
    Path(outputdir).mkdir(parents=True, exist_ok=True)
    contrast_name = '__' + '_'.join(str(e) for e in contrast)
    res.save(outputdir / (f'{filename}_Contrast{contrast_name}_ave.fif'), overwrite=True)
    return res

subjects_list = [
    '04','06','07','08','09','13','14','15','16','18','19',
    '20','21','22','23','24','25','26','27','28','29','30','32'] # List of subject to open

evokeds_contrast = pqdm(subjects_list, compute_contrast, n_jobs=N_JOBS)

  0%|          | 0/23 [00:00<?, ?it/s]

**** sub_ 04 : Reading epochs


/var/folders/97/27l9b459769fxdczmngx97xr0000gn/T/ipykernel_17968/3204102610.py:20: RuntimeWarning: converting "nave" to integer before saving evoked; this can have a minor effect on the scale of source estimates that are computed using "nave".
  res.save(outputdir / (f'{filename}_Contrast{contrast_name}_ave.fif'), overwrite=True)


**** sub_ 06 : Reading epochs


/var/folders/97/27l9b459769fxdczmngx97xr0000gn/T/ipykernel_17968/3204102610.py:20: RuntimeWarning: converting "nave" to integer before saving evoked; this can have a minor effect on the scale of source estimates that are computed using "nave".
  res.save(outputdir / (f'{filename}_Contrast{contrast_name}_ave.fif'), overwrite=True)


...

/var/folders/97/27l9b459769fxdczmngx97xr0000gn/T/ipykernel_17968/3204102610.py:20: RuntimeWarning: converting "nave" to integer before saving evoked; this can have a minor effect on the scale of source estimates that are computed using "nave".
  res.save(outputdir / (f'{filename}_Contrast{contrast_name}_ave.fif'), overwrite=True)


**** sub_ 30 : Reading epochs


/var/folders/97/27l9b459769fxdczmngx97xr0000gn/T/ipykernel_17968/3204102610.py:20: RuntimeWarning: converting "nave" to integer before saving evoked; this can have a minor effect on the scale of source estimates that are computed using "nave".
  res.save(outputdir / (f'{filename}_Contrast{contrast_name}_ave.fif'), overwrite=True)


**** sub_ 32 : Reading epochs


/var/folders/97/27l9b459769fxdczmngx97xr0000gn/T/ipykernel_17968/3204102610.py:20: RuntimeWarning: converting "nave" to integer before saving evoked; this can have a minor effect on the scale of source estimates that are computed using "nave".
  res.save(outputdir / (f'{filename}_Contrast{contrast_name}_ave.fif'), overwrite=True)


In [8]:
grand_average = mne.grand_average(evokeds_contrast)
grand_average.plot_joint();

In [ ]:
# threshold = None
# cluster_stats, data_array_ch_type, ch_type, times = stat_func.run_cluster_permutation_test_1samp(
#     evokeds_contrast, tmin=0, tmax=0.25, ch_type='mag', nperm=2 ** 12,
#     threshold=threshold, n_jobs=N_JOBS, tail=0
# )


In [17]:
# compute adjacency
ch_type = 'mag'
adjacency, ch_names = mne.channels.find_ch_adjacency(evokeds_contrast[0].info, ch_type)
tmin, tmax = 0, None

# subset of the data, as array
data = np.array(
    [
        np.transpose(
            evoked.copy().pick(ch_type).crop(tmin, None).data
        )
        for evoked in evokeds_contrast
    ]
)
times = evokeds_contrast[0].copy().crop(tmin, tmax).times
info = evokeds_contrast[0].copy().pick(ch_type).info
data.shape, adjacency.shape, times.shape

((23, 32, 102), (102, 102), (32,))

In [14]:
from mne.stats import permutation_cluster_1samp_test, spatio_temporal_cluster_1samp_test

n_permutations = 2**12
tail = 0
threshold = None

# stat func
# cluster_stats = permutation_cluster_1samp_test(data_array_chtype, threshold=threshold, n_jobs=n_jobs, verbose=True,
#                                                   tail=tail, n_permutations=2**12, adjacency=adjacency,
#                                                   out_type='indices')

cluster_stats = spatio_temporal_cluster_1samp_test(
    data,
    threshold=threshold,
    n_jobs=N_JOBS,
    tail=tail,
    n_permutations=n_permutations,
    adjacency=adjacency,
    verbose=True,
)

T_obs, clusters, p_values, _ = cluster_stats

Using a threshold of 2.073873
stat_fun(H1): min=-3.585704 max=4.689721
Running initial clustering
Found 27 clusters
Permuting 4095 times...


100%|██████████|  : 4095/4095 [00:24<00:00,  166.45it/s]

Computing cluster p-values
Done.


In [22]:
p_threshold = 0.2
good_cluster_inds = np.where(p_values < p_threshold)[0]
good_cluster_inds

array([5])

In [16]:
plt.plot(times, T_obs);

In [18]:

cluster_info = stat_func.extract_info_cluster(
    cluster_stats, p_threshold, times,
    info, data, ch_type)

We found 102 positions for ch_type mag


In [19]:

stat_func.plot_clusters(
    cluster_info, ch_type, T_obs_max=3., fname='', figname_initial='',
    filter_smooth=False, outfile=None)

Saving _clust_1.svg


True